In [128]:
import tatqa_utils
import pandas as pd
import requests

import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.exp13.db"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
#model_name = "codellama-7b-hf"

In [72]:
def infer(triton_url, prompt):

        payload = {
            "text_input": prompt,
            "max_tokens": 500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
        )
        res = ret.json()
        #print(res)
        query_response = res["text_output"]
        return query_response
    
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", "HI")

'HI-TECH\nThe HI-TECH is a 1960s-era, 19-foot, 10-inch, 2-stroke, 2-cylinder, 100 horsepower, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc,'

In [77]:
import triton_utils
model_name = "codellama-7b-instruct-hf"
os.environ["TRITON_LLM_ENDPOINT"] =  f"http://172.22.214.120:28800/v2/models/{model_name}/generate"
llm = triton_utils.TritonLLM3()
llm.invoke("So?32")

'So?320000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [87]:
model_name = "codellama-7b-instruct-hf"
os.environ["TRITON_LLM_ENDPOINT"] =  f"http://172.22.214.120:28800/v2/models/{model_name}/generate"
llm = triton_utils.TritonLLM3()

def gen_code_infer(model_url, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    print(prompt)    
    res = infer(model_url, prompt)
    #res = llm.invoke(prompt)
    #print(res)#
    code = res.replace('```python','').replace('```','')
    code = code.replace('Answer:','')
    code = code.replace(prompt,'').strip()
    
    print("S", code, "@")
    return code


def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    print(prompt)    
    #res = infer(model_url, prompt)
    res = llm.invoke(prompt)
    #print(res)#
    code = res.replace('```python','').replace('```','')
    code = code.replace('Answer:','')
    code = code.replace(prompt,'').strip()
    
    print("S", code, "@")
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
code = gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)
#code = gen_code(llm, question, table)
exec_code(code, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
S def run():
    return (592 - 519) / 519 * 100, 2 @


(14.065510597302506, 2)

In [123]:
item = devdf.iloc[0]
table = item['table']['table']
question =  item['questions'][4] ['question'] 
gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What is the change in Other in 2019 from 2018? 
 Table: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
S def run():
    return (44.1 - 56.7, 1) @


'def run():\n    return (44.1 - 56.7, 1)'

In [124]:
item = devdf.iloc[0]
table = item['table']['table']
question =  item['questions'][5]['question'] 
gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What is the percentage change in Other in 2019 from 2018? 
 Table: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
S def run():
    # Your code here
    return (0, 0) @


'def run():\n    # Your code here\n    return (0, 0)'

In [121]:
question = "What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?"
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
S def run():
    return (592 - 519) / 519 * 100, 2 @


'def run():\n    return (592 - 519) / 519 * 100, 2'

In [147]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in devdf.iterrows():
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from'] : 
        #if True: 
            code = gen_code(llm, q['question'], table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            elif  isinstance(r, float):
                (pred_value, pred_unit) = (r, None)
            elif  isinstance(r, int):
                (pred_value, pred_unit) = (r, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, item['table'], err))

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What is the change in Other in 2019 from 2018? 
 Table: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
S def run():
    return (44.1 - 56.7, 1) @
Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What is the percentage change in Other in 2019 from 2018? 
 Table:

In [133]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

special case
special case
0.0 0.0 0.0


In [139]:
[ (ans['answer'], pred, pred_scale) for ans, pred, pred_scale, _,_, _,_ in res]

[(-12.6, -12.600000000000001, 1),
 (-22.22, 0, 0),
 (-94, -2, 1),
 (-12.14, 0.060902540766021994, 100),
 (2.1, 0.25, -0.8999999999999999),
 (6.67, 0.06666666666666674, 1),
 (-8.11, -8.108108108108116, 2),
 (172, 166, 2019),
 (50.5, 57, '€m'),
 (121.5, 1, 2),
 (26.82, 26.81761528874117, 2),
 (25.1, 0.2510052777079668, 100),
 (2.98, -95.00502638853983, 2),
 (2.93, 0.43014301430143015, None),
 (0.87, None, None),
 (-361, 52, 1),
 (73, 12581, 1),
 (14.07, 14.065510597302506, 2),
 (4227.5, None, None),
 (3680, None, None),
 (547.5, None, None),
 (3728, None, None),
 (-16458, None, None),
 (154, 154, 1),
 (104.05, 104.05405405405406, 2),
 (1.57, 1.5, 1.0),
 (1.9, 2.7, 'percent'),
 (18, 18, 'increase'),
 (399.33, 453, 1),
 (88.06, 88.05970149253731, '%'),
 (0.2, 1.0, 1.0),
 (40, 39.99999999999999, 2),
 (0.51, None, None),
 (0.7000000000000001, 1.8971238938053097, 1),
 (13237.4, None, None),
 (35796, None, None),
 (64509, 45678, 1000),
 (62740, None, None),
 (14212, None, None),
 (-67.6, None,

In [146]:
good = 0
for ans, pred, pred_scale, _,_, _,_ in res:
    llimit = ans['answer']*0.90
    ulimit = ans['answer']*1.10
    
    if pred  is not None and not isinstance(pred, tuple) and  llimit < pred and pred < ulimit:
        good = good + 1
print (good/len(res))

0.17692307692307693
